In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import system.robot as robotSystem
import system.utils.dynamic_properties as part_properties
import numpy as np
import casadi as ca

In [2]:

alpha = robotSystem.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf','reach_alpha_5',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"

joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
base_T0 = [0.190, 0.000, -0.120, 3.142, 0.000, 0.000] #floating base mount
base_T1 = [0.0, 0.000, 0.0, 0.0, 0.000, 0.000] #fixed base mount
base_origin = [0.0, 1.0, 0, 0, 0, 0]
kinematic_dict, K, P, L, D, C, g, qdd, joint_torque = alpha.build_model(root, tip, floating_base=True)
c_parms, m_params, I_params, vec_g, q, q_dot, q_dotdot, tau, base_pose, world_pose = kinematic_dict['parameters']
n_joints = kinematic_dict['n_joints']
N = alpha.get_N
D_dot_2C = alpha.get_D_dot_2C
H = alpha.get_total_energy
H_dot = alpha.get_total_power

In [3]:
L.size(), K.size(), P.size(), D.size(), g.size()

((1, 1), (1, 1), (1, 1), (4, 4), (4, 1))

In [4]:
D_f = ca.Function('inertia_matrix', [ca.vertcat(*c_parms), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               vec_g,
                               q,
                               q_dot,
                               base_pose,
                               world_pose ],[K, D , g, D_dot_2C, N, P])

In [5]:
joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
q_sample = np.array([5.23716914, 1.52230767, 0.08661294, 4.24328965])
c_sample = [5e-3 ,-1e-3 ,16e-3,
    5e-3, -1e-3, 16e-3,
    5e-3, -1e-3, 16e-3,
    5e-3, -1e-3, 16e-3]
m_sample = [0.341, 0.341, 0.341, 0.341]
I_sample = [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0]
g_vec_sample = [0, 0, -9.81]
qdot_sample = [0.1, 0.2, 0.3, 0.4]
kinetic_energy, robot_inertia_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f(c_sample,
                                                                                                        m_sample,
                                                                                                        I_sample,
                                                                                                        g_vec_sample,
    q_sample,
    qdot_sample,
    base_T0,
    base_origin
    )

kinetic_energy, robot_inertia_matrix, potential_energy

(DM(0.00179993),
 DM(
 [[0.0529475, -3.10168e-05, 1.0251e-05, 0.00155671], 
  [-3.10168e-05, 0.0415245, -0.0185338, 2.28575e-05], 
  [1.0251e-05, -0.0185338, 0.0211285, 1.21337e-05], 
  [0.00155671, 2.28575e-05, 1.21337e-05, 0.0100089]]),
 DM(2.51301))

In [6]:
mrc_lump, I_lump = alpha._sys_id_lump_parameters(kinematic_dict)

mrc_I_i_fun = ca.Function('mrc_I_i', [ca.vertcat(*c_parms), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               q,
                               base_pose,
                               world_pose ], [ca.vertcat(*mrc_lump), ca.vertcat(*I_lump)])

K_reg, P_reg, D_reg, m_id_list, m_rci_id_list, I_id_list = alpha._build_sys_id_regressor(kinematic_dict)
reg_fun = ca.Function('reg', [ca.vertcat(*m_id_list), ca.vertcat(*m_rci_id_list),ca.vertcat(*I_id_list), vec_g, q,q_dot,base_pose,world_pose],[K_reg, D_reg, P_reg])

mrc_lump_sample , I_lump_sample =  mrc_I_i_fun(c_sample, m_sample, I_sample ,q_sample, base_T0, base_origin)
reg_fun(m_sample, mrc_lump_sample, I_lump_sample, g_vec_sample, q_sample, qdot_sample, base_T0, base_origin)

(DM(0.00179993),
 DM(
 [[0.0529475, -3.10168e-05, 1.0251e-05, 0.00155671], 
  [-3.10168e-05, 0.0415245, -0.0185338, 2.28575e-05], 
  [1.0251e-05, -0.0185338, 0.0211285, 1.21337e-05], 
  [0.00155671, 2.28575e-05, 1.21337e-05, 0.0100089]]),
 DM(2.51301))

In [7]:
for i in range(10000):
    joint_min = np.array([1.00, 0.01, 0.01, 0.01])
    joint_max = np.array([5.50, 3.40, 3.40, 5.70])
    q_sample = np.random.uniform(joint_min, joint_max)

    kinetic_energy, robot_inertia_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f([5e-3 ,-1e-3 ,16e-3,
        5e-3, -1e-3, 16e-3,
        5e-3, -1e-3, 16e-3,
        5e-3, -1e-3, 16e-3],
        [0.341, 0.341, 0.341, 0.341],
        [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0],
        [0, 0, -9.81],
        q_sample,
        [0.0, 0.0, 0.0, 0.1],
        [0.190, 0.000, -0.120, 3.142, 0.000, 0.000],
        [0.0, 1.0, 0, 0, 0, 0])

    checks = part_properties.is_spd_strict(robot_inertia_matrix) and part_properties.is_skew_symmetric(N_matrix) and np.allclose(D_dot_2C_matrix, N_matrix)
    if not checks:
        raise ValueError(f"check failed for q:{q_sample}")
